#### Thyroid_Disease_Predicition
Problem Statement:

Thyroid disease is a common cause of medical diagnosis and prediction, with an onset that is difficult to forecast in medical research. The thyroid gland is one of our body's most vital organs. Thyroid hormone releases are responsible for metabolic regulation. Hyperthyroidism and hypothyroidism are one of the two common diseases of the thyroid that releases thyroid hormones in regulating the rate of body's metabolism.

The main goal is to predict the estimated risk on a patient's chance of obtaining thyroid disease or not.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score


In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
from imblearn.over_sampling import RandomOverSampler

In [4]:
from imbalance import balance
from data_preproces import preprocessing
from logger import app_logger

In [5]:
pd.set_option('display.max_rows',None)

In [6]:
data=pd.read_csv("hypothyroid.csv")
data.head(5)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,?,SVHC,negative
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,?,f,?,f,?,other,negative
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,?,other,negative
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,?,f,?,f,?,other,negative
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,?,SVI,negative


In [7]:
pd.set_option('display.max_columns',None)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3772 non-null   object
 1   sex                        3772 non-null   object
 2   on_thyroxine               3772 non-null   object
 3   query_on_thyroxine         3772 non-null   object
 4   on_antithyroid_medication  3772 non-null   object
 5   sick                       3772 non-null   object
 6   pregnant                   3772 non-null   object
 7   thyroid_surgery            3772 non-null   object
 8   I131_treatment             3772 non-null   object
 9   query_hypothyroid          3772 non-null   object
 10  query_hyperthyroid         3772 non-null   object
 11  lithium                    3772 non-null   object
 12  goitre                     3772 non-null   object
 13  tumor                      3772 non-null   object
 14  hypopitu

In [9]:
data.describe()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
count,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772
unique,94,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,288,2,70,2,242,2,147,2,235,1,1,5,4
top,59,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,?,t,?,t,?,t,?,t,?,f,?,other,negative
freq,95,2480,3308,3722,3729,3625,3719,3719,3713,3538,3535,3754,3738,3676,3771,3588,3403,369,3003,769,3541,231,3385,387,3387,385,3772,3772,2201,3481



## Data Preprocessing

seems some of the columns has '?' as empty values

In [10]:
data.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Class
0,41,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,?,SVHC,negative
1,23,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,?,f,?,f,?,other,negative
2,46,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,?,t,109,t,0.91,t,120,f,?,other,negative
3,70,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,?,f,?,f,?,other,negative
4,70,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,?,SVI,negative


In [11]:
data=data.drop(["TBG","TSH_measured","T3_measured","TT4_measured","T4U_measured","FTI_measured","TBG_measured"],axis=1)

In [12]:
data.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,Class
0,41,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,1.3,2.5,125,1.14,109,SVHC,negative
1,23,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,4.1,2,102,?,?,other,negative
2,46,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,0.98,?,109,0.91,120,other,negative
3,70,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,0.16,1.9,175,?,?,other,negative
4,70,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,0.72,1.2,61,0.87,70,SVI,negative


In [13]:
pre=preprocessing()

In [14]:
pre.preprocessing(data,'pre_data.csv')

In [15]:
file=pd.read_csv('pre_data.csv')
file.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,Class
0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,125.0,1.0,109.0,0.0,0.0
1,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,102.0,1.0,102.0,1.0,0.0
2,46.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,109.0,1.0,120.0,1.0,0.0
3,70.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,175.0,1.0,160.0,1.0,0.0
4,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,61.0,1.0,70.0,2.0,0.0


In [16]:
file[['Class']].value_counts()

Class
0.0      3481
1.0       194
2.0        95
3.0         2
dtype: int64

In [17]:
file["Class"] = file["Class"].apply(lambda value : 1 if value >= 1 else 0)
file['Class'].value_counts()

0    3481
1     291
Name: Class, dtype: int64

"TBG" has 3772 as null value it is good to drop and some columns too and unwanted columns
"THS_measured","TT4_measured","T4U_measured","FTI_measured","TBG_measured"

In [18]:
file.columns

Index(['age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
       'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
       'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH', 'T3', 'TT4', 'T4U',
       'FTI', 'referral_source', 'Class'],
      dtype='object')

"?" is replaced with NAN Sucessfully

In [19]:
file.shape

(3772, 23)

In [20]:
file.isna().sum()

age                          0
sex                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              0
I131_treatment               0
query_hypothyroid            0
query_hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH                          0
T3                           0
TT4                          0
T4U                          0
FTI                          0
referral_source              0
Class                        0
dtype: int64

## Replacing Categorical to numbers

As 'class' has some imbalance we can add all Hypothyroid as (1) one and negative as (0) seperate output

In [21]:
y=file[['Class']]

In [22]:
y.value_counts()

Class
0        3481
1         291
dtype: int64

In [23]:
x=file.drop('Class',axis=1)

In [24]:
x.shape

(3772, 22)

In [25]:
x.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source
0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,125.0,1.0,109.0,0.0
1,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,102.0,1.0,102.0,1.0
2,46.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,109.0,1.0,120.0,1.0
3,70.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,175.0,1.0,160.0,1.0
4,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,61.0,1.0,70.0,2.0


In [26]:
x['referral_source'].unique()

array([0., 1., 2., 3., 4.])

In [27]:
bal=balance()

In [28]:
bal.balance_imbal(x,y)

In [29]:
x_=pd.read_csv('balance_x_train')
y_=pd.read_csv('balance_y_train')

In [30]:
y_.value_counts()

Class
0        2791
1        2791
dtype: int64

### feature selection

In [31]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(x,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

#concat two dataframes for better visualization 

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(11,'Score'))  #print 10 best features

                Specs         Score
16                TSH  76867.572631
20                FTI   3758.532029
18                TT4   3586.106964
17                 T3     37.629927
9   query_hypothyroid     26.338754
2        on_thyroxine     21.736074
1                 sex      5.726393
6            pregnant      4.430623
21    referral_source      3.154831
15              psych      2.929734
12             goitre      2.842287


### model selection 

In [32]:
rfc=RandomForestClassifier( )

forest_params = [{'max_depth': list(range(10, 15)), 'max_features': list(range(0,14))}]


In [33]:
x_train,x_test,y_train,y_test=train_test_split(x_,y_,train_size=0.8, random_state=128)

In [34]:
t=featureScores.nlargest(11,'Score')['Specs']
x_train[t].head()

,TSH,FTI,TT4,T3,query_hypothyroid,on_thyroxine,sex,pregnant,referral_source,psych,goitre
3031,160.0,9.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
16,0.0,157.0,172.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2108,1.0,118.0,117.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3265,20.0,71.0,111.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5339,8.0,61.0,59.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV (rfc, forest_params, cv = 5, scoring='accuracy')
clf.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [10, 11, 12, 13, 14],
                          'max_features': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                           12, 13]}],
             scoring='accuracy')

In [36]:
print(clf.best_params_)

print(clf.best_score_)

{'max_depth': 14, 'max_features': 13}
0.9894736842105264


In [37]:
final_x=x_train[t]

final_x_train=final_x.drop(['referral_source','goitre'], axis=1)


In [38]:
x_test=pd.read_csv('balance_x_test')
y_test=pd.read_csv('balance_y_test')

In [39]:
final_x_test=x_test[t]

In [40]:
columns=final_x_train.columns

In [41]:
final_model=RandomForestClassifier(n_estimators=15, criterion='gini', max_depth=14, max_features=9)
final_model.fit(final_x_train,y_train)
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
y_pred=final_model.predict(x_test[columns])
cf_matrix=np.array(confusion_matrix(y_test,y_pred))

In [42]:
print(cf_matrix)

[[679  11]
 [ 18 672]]


In [43]:
y_score=final_model.predict_proba(x_test[columns])


In [44]:
print(y_score)

[[1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 ...
 [0.         1.        ]
 [0.06666667 0.93333333]
 [0.06666667 0.93333333]]


In [45]:
from sklearn.model_selection import cross_val_score
cross_val_score(final_model,final_x_test,y_test,cv=3,scoring='roc_auc_ovr').mean()

0.9955923125393825

In [46]:
cross_val_score(final_model,final_x_test,y_test,cv=3,scoring='roc_auc_ovo').mean()

0.9956238185255198

In [47]:
x_test=final_x_test[columns]

In [48]:
y_score=final_model.predict_proba(x_test)

In [49]:
import pickle 

In [50]:
import joblib

In [51]:
joblib.dump(final_model,'model.pkl')

['model.pkl']

In [52]:
final_x_train.head()

,TSH,FTI,TT4,T3,query_hypothyroid,on_thyroxine,sex,pregnant,psych
3031,160.0,9.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,157.0,172.0,2.0,0.0,1.0,0.0,0.0,0.0
2108,1.0,118.0,117.0,2.0,0.0,0.0,1.0,0.0,0.0
3265,20.0,71.0,111.0,2.0,0.0,0.0,0.0,0.0,0.0
5339,8.0,61.0,59.0,2.0,0.0,0.0,0.0,0.0,0.0


In [53]:
pre=joblib.load('model.pkl')

In [54]:
pre.predict([[12.0,87.0,99.0,2.0,1.0,0.0,0.0,0.0,0.0,]])

array([1], dtype=int64)

In [58]:
from model import Model_Finder
model=Model_Finder()

In [59]:
x_filter=x_[t]

In [60]:
model.get_best_params_for_random_forest(x_filter, y_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10;, score=0.982 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10;, score=0.962 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10;, score=0.966 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10;, score=0.974 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=50;, score=0.975 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=50;, score=0.969 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=50;, score=0.971 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=aut

[CV 5/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=100;, score=0.979 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=130;, score=0.981 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=130;, score=0.977 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=130;, score=0.980 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=130;, score=0.975 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=3, max_features=log2, n_estimators=130;, score=0.979 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=10;, score=0.977 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=10;, score=0.979 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=10;, score=0.954 total time=   

[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=50;, score=0.982 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100;, score=0.982 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100;, score=0.983 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100;, score=0.986 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100;, score=0.981 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100;, score=0.983 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=130;, score=0.979 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=130;, score=0.976 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=130;, score=

RandomForestClassifier(criterion='entropy', max_depth=3, max_features='log2')

In [62]:
model.get_best_model(x_filter,y_,final_x_test, y_test)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1;, score=0.987 total time=   0.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1;, score=0.983 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1;, score=0.982 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1;, score=0.986 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=1;, score=0.982 total time=   0.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2;, score=0.989 total time=   0.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2;, score=0.983 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2;, score=0.985 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=10, n_neighbors=4, p=2;, score=0.983 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, lea

[CV 2/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=10, p=2;, score=0.970 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=10, p=2;, score=0.970 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=10, p=2;, score=0.973 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=10, p=2;, score=0.959 total time=   0.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=11, p=1;, score=0.977 total time=   0.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=11, p=1;, score=0.974 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=11, p=1;, score=0.975 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=11, p=1;, score=0.975 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=11, p=1;, score=0.968 total time=   0.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=17, n_neighbors=11, p=2;, score=0.975 total tim

[CV 2/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=1;, score=0.980 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=1;, score=0.978 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=1;, score=0.979 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=1;, score=0.970 total time=   0.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=2;, score=0.983 total time=   0.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=2;, score=0.975 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=2;, score=0.975 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=2;, score=0.978 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=8, p=2;, score=0.964 total time=   0.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=28, n_neighbors=10, p=1;, score=0.980 total time=   0.0s

[CV 1/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=4, p=2;, score=0.989 total time=   0.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=4, p=2;, score=0.983 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=4, p=2;, score=0.984 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=4, p=2;, score=0.983 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=4, p=2;, score=0.981 total time=   0.0s
[CV 1/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=5, p=1;, score=0.986 total time=   0.0s
[CV 2/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=5, p=1;, score=0.980 total time=   0.0s
[CV 3/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=5, p=1;, score=0.978 total time=   0.0s
[CV 4/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=5, p=1;, score=0.984 total time=   0.0s
[CV 5/5] END algorithm=ball_tree, leaf_size=35, n_neighbors=5, p=1;, score=0.978 total time=   0.0s


[CV 1/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=1;, score=0.977 total time=   0.0s
[CV 2/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=1;, score=0.973 total time=   0.0s
[CV 3/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=1;, score=0.974 total time=   0.0s
[CV 4/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=1;, score=0.978 total time=   0.0s
[CV 5/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=1;, score=0.967 total time=   0.0s
[CV 1/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=2;, score=0.975 total time=   0.0s
[CV 2/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=2;, score=0.968 total time=   0.0s
[CV 3/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=2;, score=0.967 total time=   0.0s
[CV 4/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=2;, score=0.971 total time=   0.0s
[CV 5/5] END algorithm=kd_tree, leaf_size=10, n_neighbors=11, p=2;, score=0.958 total time=   0.0s
[CV 1/5] E

...

[CV 4/5] END algorithm=brute, leaf_size=28, n_neighbors=5, p=2;, score=0.981 total time=   0.0s
[CV 5/5] END algorithm=brute, leaf_size=28, n_neighbors=5, p=2;, score=0.978 total time=   0.0s
[CV 1/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=1;, score=0.981 total time=   0.0s
[CV 2/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=1;, score=0.978 total time=   0.0s
[CV 3/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=1;, score=0.977 total time=   0.0s
[CV 4/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=1;, score=0.978 total time=   0.0s
[CV 5/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=1;, score=0.968 total time=   0.0s
[CV 1/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=2;, score=0.983 total time=   0.0s
[CV 2/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=2;, score=0.974 total time=   0.0s
[CV 3/5] END algorithm=brute, leaf_size=28, n_neighbors=8, p=2;, score=0.974 total time=   0.0s
[CV 4/5] END algorithm=brute, leaf_size=

[CV 5/5] END algorithm=brute, leaf_size=35, n_neighbors=4, p=1;, score=0.981 total time=   0.0s
[CV 1/5] END algorithm=brute, leaf_size=35, n_neighbors=4, p=2;, score=0.988 total time=   0.0s
[CV 2/5] END algorithm=brute, leaf_size=35, n_neighbors=4, p=2;, score=0.982 total time=   0.0s
[CV 3/5] END algorithm=brute, leaf_size=35, n_neighbors=4, p=2;, score=0.983 total time=   0.0s
[CV 4/5] END algorithm=brute, leaf_size=35, n_neighbors=4, p=2;, score=0.983 total time=   0.0s
[CV 5/5] END algorithm=brute, leaf_size=35, n_neighbors=4, p=2;, score=0.981 total time=   0.0s
[CV 1/5] END algorithm=brute, leaf_size=35, n_neighbors=5, p=1;, score=0.982 total time=   0.0s
[CV 2/5] END algorithm=brute, leaf_size=35, n_neighbors=5, p=1;, score=0.979 total time=   0.0s
[CV 3/5] END algorithm=brute, leaf_size=35, n_neighbors=5, p=1;, score=0.977 total time=   0.0s
[CV 4/5] END algorithm=brute, leaf_size=35, n_neighbors=5, p=1;, score=0.979 total time=   0.0s
[CV 5/5] END algorithm=brute, leaf_size=

[CV 5/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=100;, score=0.978 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=130;, score=0.971 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=130;, score=0.974 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=130;, score=0.981 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=130;, score=0.974 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_features=log2, n_estimators=130;, score=0.979 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=10;, score=0.976 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=10;, score=0.978 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=10;, score=0.985 total time=   0.0s
[CV 

[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=100;, score=0.973 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=100;, score=0.973 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=100;, score=0.973 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=100;, score=0.966 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=100;, score=0.978 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=130;, score=0.976 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=130;, score=0.972 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=130;, score=0.975 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=130;, score

('RandomForest',
 RandomForestClassifier(criterion='entropy', max_depth=3, max_features='log2'))